In [45]:
import pandas as pd
import numpy as np
import re

import unicodedata

# TO-DO

- que se tome en cuenta si los primeros 2 datos son cm (actualmente se asume que son pulgadas)
  - ej: 27x25 cm (10x5 in)

# limpieza de datos

In [2]:
def get_sold(df):
    
    sold = df[df['precio_venta'].str.contains('upcoming') == False]
    sold = sold[sold['precio_venta'].str.contains('live now') == False]
    sold = sold[sold['precio_venta'].str.contains('results coming soon') == False]
    sold = sold[sold['precio_venta'].str.contains('not sold') == False]
    
    return sold

In [3]:
df = pd.read_csv('../datasets/MutualArtLimpio.csv')

df = get_sold(df)

# Clean dimensiones
(height va primero)

idea 1:
- detectar si está en inches o cm (eliminar los que no incluyen medidas)
- extraer fracciones
- extraer los primeros 2 números


In [4]:

def fraction_char_replace(s):
    '''
    Reemplaza las fracciones representadas como un solo caracter y las convierte en decimales
    ej: ¾, ½ se convierten en .75 y .5
    Args: 
        s(str)- string de las dimensiones de una obra
    '''
    
    for c in s:
        try:
            name = unicodedata.name(c)
        except ValueError:
            continue
        if name.startswith('VULGAR FRACTION'):
            normalized = unicodedata.normalize('NFKC', c)
            numerator, _slash, denominator = normalized.partition('⁄')
            
            frac = int(numerator)/int(denominator)
            s = s.replace(c, str(frac))
            #yield c, numerator, denominator
            
            
    return s
            
#TO-DO
#- encontrar source de la fuente

In [5]:
def fraction_str_replace(s):
    '''
    Se asume que dentro de cada registro, se encuentra un 
    Formas de detectat una fraccion de la forma a/b:
    
    num a/b
    
    aaa/b
    
    num-a/b
    
    num.a/b
    
    ojo: existe .groups()
    '''
    if(re.search(' [0-9]?[0-9]/[0-9]?[0-9]', s)):  #tipo num a/b
        
        #es un ciclo porque se debe de tratar cada fraccion en todo el string
        for m in re.findall('([0-9]?[0-9])/([0-9]?[0-9])',s): #lista de tuplas para cada fracción
            num, denom = int(m[0]), int(m[1])
            frac = num/denom
            frac = re.sub('0','',str(frac))
            
            pattern = ' '+str(num)+'/'+str(denom)
            
            s = re.sub(pattern, frac,s)
        
    elif(re.search('[0-9][0-9]{2}?/[0-9][0-9]?',s)): #tipo aaa/b
        
        for m in re.findall('([0-9][0-9][0-9])/([0-9][0-9]?)',s): #lista de tuplas para cada fracción
            num, denom = int(m[0]), int(m[1])
            frac = num/denom
            pattern = str(num)+'/'+str(denom)
            
            s = re.sub(pattern, str(frac),s)
        
    elif(re.search('[0-9][0-9]?-[0-9][0-9]?/[0-9]?[0-9]',s)):
        #tipo num-a/b
        
        for m in re.findall('([0-9][0-9]?[0-9]?)/([0-9][0-9]?)',s): #lista de tuplas para cada fracción
            num, denom = int(m[0]), int(m[1])
            frac = num/denom
            frac = re.sub('0','',str(frac))
            pattern = '-'+str(num)+'/'+str(denom)
            
            s = re.sub(pattern, frac,s)
        
    elif(re.search('[0-9]\.[0-9][0-9]?/[0-9]?[0-9]',s)):
        #tipo num.a/b
        
        for m in re.findall('([0-9][0-9]?[0-9]?)/([0-9][0-9]?)',s): #lista de tuplas para cada fracción
            num, denom = int(m[0]), int(m[1])
            frac = num/denom
            frac = re.sub('0\.','',str(frac))
            pattern = str(num)+'/'+str(denom)
            
            s = re.sub(pattern, frac,s)
        
    else: 
        resp = 'no se detectó'
    
    return s

#TO-DO:
#simplificar código
#talvez se pueden hacer menos ifs
#mejorar comentarios

In [6]:
#test de la funcion anterior

#fracciones = df[df['dimensiones'].str.contains('[0-9]\/[0-9]') == True].copy()#.head(5)
#fracciones['tipo_frac'] = fracciones.dimensiones.apply(lambda x: fraction_str_replace(x))

In [7]:
#fracciones
#ejem = fracciones.iloc[2]['dimensiones']

In [8]:
def is_inches(s):
    '''
    
    '''
    no_measures_ids = ['various sizes', 'dimensions variable', 'property from the estate of bernard chappard\xa0 ...']
    inches_ids = ['in', '"', '\'\'', '″',  'a']
    cm_ids = ['cm', 'mm', 'm']
    
    if any(keyword in s for keyword in inches_ids):
        is_inch = 1
    elif any(keyword in s for keyword in cm_ids):
        is_inch = 0
    elif any(keyword in s for keyword in no_measures_ids):
        is_inch = -1
    else: 
        is_inch = 1 #if no imformation is given of it's measurements, it is assumed that the measures are in inches
        #is_inch = 2 #in case if a problem is detected and some rows need to be flagged
    
    return is_inch

In [40]:
def extract_numbers(df):
    '''
    Se extrae los primeros 2 valores numéricos que se encuentre y se pasan cm si son pulgadas. Se asumen 3 cosas:
    
    - que no hay obras de 3 dimensiones (esculturas, etc...)
    - si vienen las medidas 2 veces, primero se observan las pulgadas
    - 
    '''
    df['height'] = df.dimensiones.str.extract('([0-9][0-9]?[0-9]?\.?[0-9]?[0-9]?[0-9]?)') #normalmente se extrae el primer match
    temp_series = df.dimensiones.str.replace('[0-9][0-9]?[0-9]?\.?[0-9]?[0-9]?[0-9]?', '',regex=True, n=1)
    df['height'] = pd.to_numeric(df['height'])
    df['width'] = temp_series.str.extract('([0-9][0-9]?[0-9]?\.?[0-9]?[0-9]?[0-9]?)')
    df['width'] = pd.to_numeric(df['width'])
    
    return df
    #que las pase como números

In [12]:
def clean_dimensiones(df):
    '''
    Limpia la columna dimensiones y agrega las columnas de height y width
    '''
    df['dimensiones'] = df.dimensiones.apply(lambda x: fraction_char_replace(x))
    df['dimensiones'] = df.dimensiones.apply(lambda x: fraction_str_replace(x))
    df['is_inch'] = df.dimensiones.apply(lambda x: is_inches(x))
    df = extract_numbers(df)
    
    return df

In [46]:
df_clean = clean_dimensiones(df)

In [44]:
ejem = df_clean  #.iloc[4]
extract_numbers(ejem)

,ID,precio estimado,precio_venta,fecha,autor,casa subasta,tipo,tecnica,dimensiones,precio estimado dolares,precio venta dolares,is_inch,height,width
862,863,700 eur,700 eur,"sep 21, 2022",jerson jimenez,"dorotheum, vienna",charity auction of the salvatorians,oil on canvas,50 x 40 cm,679.0,679.0,0,50.000,40.000
975,976,"4,000 - 4,400 eur","5,120 eur","sep 20, 2022",julio le parc,"dorotheum, vienna",modern and contemporary art,screenprint on cardboard,"41 x 41 x 5 cm, 8, 1970,",4268.0,4966.4,0,41.000,41.000
976,977,"1,000 - 1,500 gbp","1,275 gbp","sep 20, 2022",raul anguiano,bonhams knightsbridge,travel & exploration,oil on canvas,65.1 x 50.2cm (25 x 19.75in),1620.0,1377.0,1,65.100,50.200
977,978,"2,000 - 3,000 aud","1,200 aud*","sep 20, 2022",jose haro,ozbid auctions,huge modern art auction - live online,mixed media,94cm x 153cm,1950.0,780.0,0,94.000,153.000
978,979,350 - 400 chf,225 chf*,"sep 20, 2022",jorge camacho,tgp auction,pop & contemporary art,lithograph on guarro paper,56 x 76 cm56 x 76 cm10 h.10 h.,408.0,229.5,0,56.000,76.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5133,5134,"400,000 - 600,000 usd","607,500 usd","jun 02, 1999",rufino tamayo,christie's new york,"important latin american paintings, drawings &...",oil and sand on canvas,39.625 x 49.125in. (80.8 x 100cm.),600000.0,607500.0,1,39.625,49.125
5134,5135,"100,000 - 150,000 usd","217,000 usd","jun 02, 1999",rufino tamayo,christie's new york,"important latin american paintings, drawings &...",oil and sand on canvas,280.75 x 350.5in. (73 x 90.2cm.),150000.0,217000.0,1,280.750,350.500
5135,5136,25 - 200 gbp,126 gbp,"feb 25, 1999",cesar villacres,christie's south kensington,british and continental pictures,oil on canvas,15 x 12in. (39.3 x 30.5cm.),216.0,136.08,1,15.000,12.000
5136,5137,"2,500 - 3,000 gbp","2,530 gbp","dec 07, 1998",rene portocarrero,christie's south kensington,twentieth century art,oil and gouache on paper,230.75 x 130.75in. (60 x 35cm.),3240.0,2732.4,1,230.750,130.750


In [42]:
df_clean_2 = extract_numbers(df_clean)

In [43]:
df_clean_2

,ID,precio estimado,precio_venta,fecha,autor,casa subasta,tipo,tecnica,dimensiones,precio estimado dolares,precio venta dolares,is_inch,height,width
862,863,700 eur,700 eur,"sep 21, 2022",jerson jimenez,"dorotheum, vienna",charity auction of the salvatorians,oil on canvas,50 x 40 cm,679.0,679.0,0,50.000,40.000
975,976,"4,000 - 4,400 eur","5,120 eur","sep 20, 2022",julio le parc,"dorotheum, vienna",modern and contemporary art,screenprint on cardboard,"41 x 41 x 5 cm, 8, 1970,",4268.0,4966.4,0,41.000,41.000
976,977,"1,000 - 1,500 gbp","1,275 gbp","sep 20, 2022",raul anguiano,bonhams knightsbridge,travel & exploration,oil on canvas,65.1 x 50.2cm (25 x 19.75in),1620.0,1377.0,1,65.100,50.200
977,978,"2,000 - 3,000 aud","1,200 aud*","sep 20, 2022",jose haro,ozbid auctions,huge modern art auction - live online,mixed media,94cm x 153cm,1950.0,780.0,0,94.000,153.000
978,979,350 - 400 chf,225 chf*,"sep 20, 2022",jorge camacho,tgp auction,pop & contemporary art,lithograph on guarro paper,56 x 76 cm56 x 76 cm10 h.10 h.,408.0,229.5,0,56.000,76.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5133,5134,"400,000 - 600,000 usd","607,500 usd","jun 02, 1999",rufino tamayo,christie's new york,"important latin american paintings, drawings &...",oil and sand on canvas,39.625 x 49.125in. (80.8 x 100cm.),600000.0,607500.0,1,39.625,49.125
5134,5135,"100,000 - 150,000 usd","217,000 usd","jun 02, 1999",rufino tamayo,christie's new york,"important latin american paintings, drawings &...",oil and sand on canvas,280.75 x 350.5in. (73 x 90.2cm.),150000.0,217000.0,1,280.750,350.500
5135,5136,25 - 200 gbp,126 gbp,"feb 25, 1999",cesar villacres,christie's south kensington,british and continental pictures,oil on canvas,15 x 12in. (39.3 x 30.5cm.),216.0,136.08,1,15.000,12.000
5136,5137,"2,500 - 3,000 gbp","2,530 gbp","dec 07, 1998",rene portocarrero,christie's south kensington,twentieth century art,oil and gouache on paper,230.75 x 130.75in. (60 x 35cm.),3240.0,2732.4,1,230.750,130.750


In [47]:
df_clean

,ID,precio estimado,precio_venta,fecha,autor,casa subasta,tipo,tecnica,dimensiones,precio estimado dolares,precio venta dolares,is_inch,height,width
862,863,700 eur,700 eur,"sep 21, 2022",jerson jimenez,"dorotheum, vienna",charity auction of the salvatorians,oil on canvas,50 x 40 cm,679.0,679.0,0,50.000,40.000
975,976,"4,000 - 4,400 eur","5,120 eur","sep 20, 2022",julio le parc,"dorotheum, vienna",modern and contemporary art,screenprint on cardboard,"41 x 41 x 5 cm, 8, 1970,",4268.0,4966.4,0,41.000,41.000
976,977,"1,000 - 1,500 gbp","1,275 gbp","sep 20, 2022",raul anguiano,bonhams knightsbridge,travel & exploration,oil on canvas,65.1 x 50.2cm (25 x 19.75in),1620.0,1377.0,1,65.100,50.200
977,978,"2,000 - 3,000 aud","1,200 aud*","sep 20, 2022",jose haro,ozbid auctions,huge modern art auction - live online,mixed media,94cm x 153cm,1950.0,780.0,0,94.000,153.000
978,979,350 - 400 chf,225 chf*,"sep 20, 2022",jorge camacho,tgp auction,pop & contemporary art,lithograph on guarro paper,56 x 76 cm56 x 76 cm10 h.10 h.,408.0,229.5,0,56.000,76.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5133,5134,"400,000 - 600,000 usd","607,500 usd","jun 02, 1999",rufino tamayo,christie's new york,"important latin american paintings, drawings &...",oil and sand on canvas,39.625 x 49.125in. (80.8 x 100cm.),600000.0,607500.0,1,39.625,49.125
5134,5135,"100,000 - 150,000 usd","217,000 usd","jun 02, 1999",rufino tamayo,christie's new york,"important latin american paintings, drawings &...",oil and sand on canvas,280.75 x 350.5in. (73 x 90.2cm.),150000.0,217000.0,1,280.750,350.500
5135,5136,25 - 200 gbp,126 gbp,"feb 25, 1999",cesar villacres,christie's south kensington,british and continental pictures,oil on canvas,15 x 12in. (39.3 x 30.5cm.),216.0,136.08,1,15.000,12.000
5136,5137,"2,500 - 3,000 gbp","2,530 gbp","dec 07, 1998",rene portocarrero,christie's south kensington,twentieth century art,oil and gouache on paper,230.75 x 130.75in. (60 x 35cm.),3240.0,2732.4,1,230.750,130.750
